# Titanic Survival Prediction - Machine Learning Modeling
# =======================================================

# ## 📋 Objetivo
# Implementar y comparar múltiples algoritmos de Machine Learning para predecir supervivencia:
# - Logistic Regression (baseline interpretable)
# - Random Forest (ensemble robusto)
# - Support Vector Machine (boundaries complejas)
# - Naive Bayes (baseline probabilístico)
# - Evaluar performance y seleccionar mejor modelo

## 1. Importación de Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                            roc_auc_score, classification_report, confusion_matrix, roc_curve)
from sklearn.preprocessing import StandardScaler
import joblib
import warnings

# Funciones helper
import sys
import os
sys.path.append('../src')
from utils.helpers import save_current_plot

In [ ]:
# Configuración
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
np.random.seed(42)

print("✅ Librerías importadas correctamente")

## 2. Carga de Datos Procesados

In [ ]:
# Cargar dataset con features engineering
df_features = pd.read_csv("../data/processed/features_engineered.csv")
df_scaled = pd.read_csv("../data/processed/features_scaled.csv")

print("🚢 DATASETS CARGADOS")
print("=" * 25)
print(f"📊 Features engineered: {df_features.shape}")
print(f"📊 Features scaled: {df_scaled.shape}")

# Verificar consistencia
print(f"\n🔍 Verificación:")
print(f"  - Mismas dimensiones: {df_features.shape == df_scaled.shape}")
print(
    f"  - Variables objetivo iguales: {df_features['Survived'].equals(df_scaled['Survived'])}"
)

print("\n📋 Variables disponibles:")
print(f"Total features: {len(df_features.columns)}")
print("\nTipos de features:")
feature_types = {
    "Original": ["Pclass", "Age", "SibSp", "Parch", "Fare", "Cabin_Known"],
    "Engineered": ["FamilySize", "IsAlone"],
    "Encoded": [
        "Sex_Encoded",
        "Embarked_Encoded",
        "AgeGroup_Encoded",
        "FareBin_Encoded",
    ],
    "Interaction": [
        col
        for col in df_features.columns
        if any(x in col for x in ["Title_", "SexPclass_", "AgeSex_"])
    ],
}

for ftype, features in feature_types.items():
    available_features = [f for f in features if f in df_features.columns]
    print(f"  - {ftype}: {len(available_features)} features")

## 3. Preparación de Datos para Modelado

In [ ]:
print("\n🎯 PREPARACIÓN DE DATOS PARA MODELADO")
print("=" * 40)

# Separar features y target
X = df_scaled.drop("Survived", axis=1)
y = df_scaled["Survived"]

print(f"📊 Matriz de features (X): {X.shape}")
print(f"🎯 Variable objetivo (y): {y.shape}")
print(f"📈 Distribución de clases: {y.value_counts().to_dict()}")
print(f"⚖️ Balance de clases: {y.mean():.3f} (38.4% supervivencia)")

### 3.1 Split Train/Validation/Test

In [ ]:
print("\n🔄 DIVISIÓN DE DATOS")
print("=" * 20)

# Split inicial: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Split train/validation: 70% train, 10% validation del total
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.125,
    random_state=42,
    stratify=y_temp,  # 0.125 de 0.8 = 0.1 del total
)

print(f"📊 Dimensiones finales:")
print(f"  - Train: {X_train.shape[0]} muestras ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  - Validation: {X_val.shape[0]} muestras ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  - Test: {X_test.shape[0]} muestras ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\n⚖️ Balance por conjunto:")
print(f"  - Train: {y_train.mean():.3f}")
print(f"  - Validation: {y_val.mean():.3f}")
print(f"  - Test: {y_test.mean():.3f}")

## 4. Implementación de Modelos Base

In [ ]:
print("\n🤖 IMPLEMENTACIÓN DE MODELOS")
print("=" * 35)

# Diccionario para almacenar modelos
models = {}
model_results = {}

### 4.1 Logistic Regression (Baseline)

In [ ]:
print("\n📊 MODELO 1: LOGISTIC REGRESSION")
print("=" * 30)

# Modelo base
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

# Predicciones
y_pred_lr_train = lr_model.predict(X_train)
y_pred_lr_val = lr_model.predict(X_val)
y_proba_lr_val = lr_model.predict_proba(X_val)[:, 1]

# Métricas
lr_metrics = {
    "train_accuracy": accuracy_score(y_train, y_pred_lr_train),
    "val_accuracy": accuracy_score(y_val, y_pred_lr_val),
    "val_precision": precision_score(y_val, y_pred_lr_val),
    "val_recall": recall_score(y_val, y_pred_lr_val),
    "val_f1": f1_score(y_val, y_pred_lr_val),
    "val_auc": roc_auc_score(y_val, y_proba_lr_val),
}

print("Métricas Logistic Regression:")
for metric, value in lr_metrics.items():
    print(f"  {metric}: {value:.4f}")

models["Logistic Regression"] = lr_model
model_results["Logistic Regression"] = lr_metrics

### 4.2 Random Forest

In [ ]:
print("\n🌲 MODELO 2: RANDOM FOREST")
print("=" * 25)

# Modelo base
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicciones
y_pred_rf_train = rf_model.predict(X_train)
y_pred_rf_val = rf_model.predict(X_val)
y_proba_rf_val = rf_model.predict_proba(X_val)[:, 1]

# Métricas
rf_metrics = {
    "train_accuracy": accuracy_score(y_train, y_pred_rf_train),
    "val_accuracy": accuracy_score(y_val, y_pred_rf_val),
    "val_precision": precision_score(y_val, y_pred_rf_val),
    "val_recall": recall_score(y_val, y_pred_rf_val),
    "val_f1": f1_score(y_val, y_pred_rf_val),
    "val_auc": roc_auc_score(y_val, y_proba_rf_val),
}

print("Métricas Random Forest:")
for metric, value in rf_metrics.items():
    print(f"  {metric}: {value:.4f}")

models["Random Forest"] = rf_model
model_results["Random Forest"] = rf_metrics

### 4.3 Support Vector Machine

In [ ]:
print("\n🎯 MODELO 3: SUPPORT VECTOR MACHINE")
print("=" * 35)

# Modelo base
svm_model = SVC(kernel="rbf", probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Predicciones
y_pred_svm_train = svm_model.predict(X_train)
y_pred_svm_val = svm_model.predict(X_val)
y_proba_svm_val = svm_model.predict_proba(X_val)[:, 1]

# Métricas
svm_metrics = {
    "train_accuracy": accuracy_score(y_train, y_pred_svm_train),
    "val_accuracy": accuracy_score(y_val, y_pred_svm_val),
    "val_precision": precision_score(y_val, y_pred_svm_val),
    "val_recall": recall_score(y_val, y_pred_svm_val),
    "val_f1": f1_score(y_val, y_pred_svm_val),
    "val_auc": roc_auc_score(y_val, y_proba_svm_val),
}

print("Métricas SVM:")
for metric, value in svm_metrics.items():
    print(f"  {metric}: {value:.4f}")

models["SVM"] = svm_model
model_results["SVM"] = svm_metrics

### 4.4 Naive Bayes

In [ ]:
print("\n🔮 MODELO 4: NAIVE BAYES")
print("=" * 25)

# Modelo base
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predicciones
y_pred_nb_train = nb_model.predict(X_train)
y_pred_nb_val = nb_model.predict(X_val)
y_proba_nb_val = nb_model.predict_proba(X_val)[:, 1]

# Métricas
nb_metrics = {
    "train_accuracy": accuracy_score(y_train, y_pred_nb_train),
    "val_accuracy": accuracy_score(y_val, y_pred_nb_val),
    "val_precision": precision_score(y_val, y_pred_nb_val),
    "val_recall": recall_score(y_val, y_pred_nb_val),
    "val_f1": f1_score(y_val, y_pred_nb_val),
    "val_auc": roc_auc_score(y_val, y_proba_nb_val),
}

print("Métricas Naive Bayes:")
for metric, value in nb_metrics.items():
    print(f"  {metric}: {value:.4f}")

models["Naive Bayes"] = nb_model
model_results["Naive Bayes"] = nb_metrics

## 5. Comparación de Modelos Base

In [ ]:
print("\n📊 COMPARACIÓN DE MODELOS BASE")
print("=" * 35)

# Crear DataFrame de comparación
comparison_df = pd.DataFrame(model_results).T
comparison_df = comparison_df.round(4)

print("Tabla de comparación:")
print(comparison_df)

# Visualización de comparación
plt.figure(figsize=(15, 10))

# Métricas de validación
val_metrics = ["val_accuracy", "val_precision", "val_recall", "val_f1", "val_auc"]
metric_names = ["Accuracy", "Precision", "Recall", "F1-Score", "AUC-ROC"]

for i, (metric, name) in enumerate(zip(val_metrics, metric_names), 1):
    plt.subplot(2, 3, i)
    values = comparison_df[metric]
    colors = ["skyblue", "lightgreen", "coral", "gold"]
    bars = plt.bar(values.index, values.values, color=colors)
    plt.title(f"{name} en Validación")
    plt.ylabel(name)
    plt.xticks(rotation=45)

    # Añadir valores en las barras
    for bar, value in zip(bars, values.values):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + 0.01,
            f"{value:.3f}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

# Gráfico de overfitting (Train vs Val Accuracy)
plt.subplot(2, 3, 6)
train_acc = comparison_df["train_accuracy"]
val_acc = comparison_df["val_accuracy"]
x = np.arange(len(models))
width = 0.35

plt.bar(x - width / 2, train_acc, width, label="Train", alpha=0.7, color="lightblue")
plt.bar(
    x + width / 2, val_acc, width, label="Validation", alpha=0.7, color="lightcoral"
)
plt.xlabel("Modelos")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.xticks(x, train_acc.index, rotation=45)
plt.legend()

plt.tight_layout()
save_current_plot("models_comparison", "../results/figures/modeling")
plt.show()

# Identificar mejor modelo base
best_metric = "val_f1"  # Métrica balanceada
best_model_name = comparison_df[best_metric].idxmax()
best_score = comparison_df.loc[best_model_name, best_metric]

print(f"\n🏆 Mejor modelo base: {best_model_name}")
print(f"📈 {best_metric}: {best_score:.4f}")

## 6. Cross-Validation para Robustez

In [ ]:
print("\n🔄 CROSS-VALIDATION PARA ROBUSTEZ")
print("=" * 35)

# Configurar CV estratificado
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_results = {}

for name, model in models.items():
    print(f"\nEvaluando {name} con 5-Fold CV:")

    # Múltiples métricas
    cv_accuracy = cross_val_score(
        model, X_temp, y_temp, cv=cv_strategy, scoring="accuracy"
    )
    cv_f1 = cross_val_score(model, X_temp, y_temp, cv=cv_strategy, scoring="f1")
    cv_auc = cross_val_score(model, X_temp, y_temp, cv=cv_strategy, scoring="roc_auc")

    cv_results[name] = {
        "accuracy_mean": cv_accuracy.mean(),
        "accuracy_std": cv_accuracy.std(),
        "f1_mean": cv_f1.mean(),
        "f1_std": cv_f1.std(),
        "auc_mean": cv_auc.mean(),
        "auc_std": cv_auc.std(),
    }

    print(f"  Accuracy: {cv_accuracy.mean():.4f} ± {cv_accuracy.std():.4f}")
    print(f"  F1-Score: {cv_f1.mean():.4f} ± {cv_f1.std():.4f}")
    print(f"  AUC-ROC: {cv_auc.mean():.4f} ± {cv_auc.std():.4f}")

# Visualización CV
cv_df = pd.DataFrame(cv_results).T

plt.figure(figsize=(15, 5))

metrics = ["accuracy", "f1", "auc"]
titles = ["Accuracy", "F1-Score", "AUC-ROC"]

for i, (metric, title) in enumerate(zip(metrics, titles), 1):
    plt.subplot(1, 3, i)
    means = cv_df[f"{metric}_mean"]
    stds = cv_df[f"{metric}_std"]

    plt.errorbar(range(len(means)), means, yerr=stds, fmt="o", capsize=5, capthick=2)
    plt.xticks(range(len(means)), means.index, rotation=45)
    plt.ylabel(title)
    plt.title(f"{title} - Cross Validation")
    plt.grid(True, alpha=0.3)

plt.tight_layout()
save_current_plot("cross_validation_results", "../results/figures/modeling")
plt.show()

## 7. Hyperparameter Tuning del Mejor Modelo

In [ ]:
print(f"\n🔧 HYPERPARAMETER TUNING: {best_model_name.upper()}")
print("=" * 50)

# Definir grids de parámetros
param_grids = {
    "Logistic Regression": {
        "C": [0.01, 0.1, 1, 10, 100],
        "penalty": ["l1", "l2"],
        "solver": ["liblinear", "saga"],
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
    },
    "SVM": {
        "C": [0.1, 1, 10, 100],
        "kernel": ["linear", "rbf"],
        "gamma": ["scale", "auto"],
    },
    "Naive Bayes": {"var_smoothing": [1e-9, 1e-8, 1e-7, 1e-6]},
}

# Grid Search del mejor modelo
best_base_model = models[best_model_name]
param_grid = param_grids[best_model_name]

print(f"Optimizando parámetros de {best_model_name}...")
print(f"Grid de búsqueda: {param_grid}")

grid_search = GridSearchCV(
    estimator=best_base_model,
    param_grid=param_grid,
    cv=cv_strategy,
    scoring="f1",
    n_jobs=-1,
    verbose=1,
)

grid_search.fit(X_temp, y_temp)

print(f"\n🎯 Mejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"\n📈 Mejor score (F1): {grid_search.best_score_:.4f}")

# Modelo optimizado
best_model_tuned = grid_search.best_estimator_

# Evaluar modelo optimizado
y_pred_tuned_train = best_model_tuned.predict(X_train)
y_pred_tuned_val = best_model_tuned.predict(X_val)
y_proba_tuned_val = best_model_tuned.predict_proba(X_val)[:, 1]

tuned_metrics = {
    "train_accuracy": accuracy_score(y_train, y_pred_tuned_train),
    "val_accuracy": accuracy_score(y_val, y_pred_tuned_val),
    "val_precision": precision_score(y_val, y_pred_tuned_val),
    "val_recall": recall_score(y_val, y_pred_tuned_val),
    "val_f1": f1_score(y_val, y_pred_tuned_val),
    "val_auc": roc_auc_score(y_val, y_proba_tuned_val),
}

print(f"\n📊 Métricas del modelo optimizado:")
for metric, value in tuned_metrics.items():
    print(f"  {metric}: {value:.4f}")

## 8. Evaluación Final en Test Set

In [ ]:
print("\n🏁 EVALUACIÓN FINAL EN TEST SET")
print("=" * 35)

# Evaluar modelo optimizado en test
y_pred_test = best_model_tuned.predict(X_test)
y_proba_test = best_model_tuned.predict_proba(X_test)[:, 1]

test_metrics = {
    "accuracy": accuracy_score(y_test, y_pred_test),
    "precision": precision_score(y_test, y_pred_test),
    "recall": recall_score(y_test, y_pred_test),
    "f1": f1_score(y_test, y_pred_test),
    "auc": roc_auc_score(y_test, y_proba_test),
}

print(f"📊 MÉTRICAS FINALES DEL MODELO OPTIMIZADO:")
for metric, value in test_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)
print(f"\n📋 Matriz de Confusión:")
print(cm)

# Reporte de clasificación
print(f"\n📝 Reporte de Clasificación:")
print(
    classification_report(
        y_test, y_pred_test, target_names=["No Sobrevivió", "Sobrevivió"]
    )
)

# Visualización de resultados finales
plt.figure(figsize=(15, 5))

# Matriz de confusión
plt.subplot(1, 3, 1)
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["No Sobrevivió", "Sobrevivió"],
    yticklabels=["No Sobrevivió", "Sobrevivió"],
)
plt.title("Matriz de Confusión - Test Set")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")

# Curva ROC
plt.subplot(1, 3, 2)
fpr, tpr, _ = roc_curve(y_test, y_proba_test)
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {test_metrics["auc"]:.3f})')
plt.plot([0, 1], [0, 1], "k--", linewidth=1, label="Random Classifier")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - Test Set")
plt.legend()
plt.grid(True, alpha=0.3)

# Distribución de probabilidades
plt.subplot(1, 3, 3)
plt.hist(
    y_proba_test[y_test == 0], bins=20, alpha=0.7, label="No Sobrevivió", color="red"
)
plt.hist(
    y_proba_test[y_test == 1], bins=20, alpha=0.7, label="Sobrevivió", color="green"
)
plt.xlabel("Probabilidad Predicha")
plt.ylabel("Frecuencia")
plt.title("Distribución de Probabilidades")
plt.legend()
plt.axvline(x=0.5, color="black", linestyle="--", alpha=0.7, label="Threshold=0.5")

plt.tight_layout()
save_current_plot("final_evaluation", "../results/figures/modeling")
plt.show()

## 9. Feature Importance Analysis

In [ ]:
print("\n🔍 ANÁLISIS DE FEATURE IMPORTANCE")
print("=" * 35)

# Feature importance (si el modelo lo soporta)
if hasattr(best_model_tuned, "feature_importances_"):
    feature_importance = pd.DataFrame(
        {"feature": X.columns, "importance": best_model_tuned.feature_importances_}
    ).sort_values("importance", ascending=False)

    print("Top 15 features más importantes:")
    print(feature_importance.head(15))

    # Visualización
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    sns.barplot(data=top_features, y="feature", x="importance", palette="viridis")
    plt.title("Top 15 Features Más Importantes")
    plt.xlabel("Importancia")
    plt.ylabel("Feature")
    plt.tight_layout()
    save_current_plot("feature_importance", "../results/figures/modeling")
    plt.show()

elif hasattr(best_model_tuned, "coef_"):
    # Para modelos lineales (Logistic Regression)
    feature_coef = pd.DataFrame(
        {
            "feature": X.columns,
            "coefficient": best_model_tuned.coef_[0],
            "abs_coefficient": np.abs(best_model_tuned.coef_[0]),
        }
    ).sort_values("abs_coefficient", ascending=False)

    print("Top 15 coeficientes más importantes:")
    print(feature_coef.head(15))

    # Visualización
    plt.figure(figsize=(12, 8))
    top_features = feature_coef.head(15)
    colors = ["red" if x < 0 else "blue" for x in top_features["coefficient"]]
    sns.barplot(data=top_features, y="feature", x="coefficient", palette=colors)
    plt.title("Top 15 Coeficientes Más Importantes")
    plt.xlabel("Coeficiente")
    plt.ylabel("Feature")
    plt.axvline(x=0, color="black", linestyle="-", alpha=0.3)
    plt.tight_layout()
    save_current_plot("feature_coefficients", "../results/figures/modeling")
    plt.show()

## 10. Resumen Comparativo Final

In [ ]:
print("\n📈 RESUMEN COMPARATIVO FINAL")
print("=" * 35)

# Crear tabla comparativa final
final_comparison = pd.DataFrame(
    {
        "Modelo": list(model_results.keys()) + [f"{best_model_name} (Optimizado)"],
        "Validation_F1": list(comparison_df["val_f1"]) + [tuned_metrics["val_f1"]],
        "Validation_Accuracy": list(comparison_df["val_accuracy"])
        + [tuned_metrics["val_accuracy"]],
        "Validation_AUC": list(comparison_df["val_auc"]) + [tuned_metrics["val_auc"]],
        "Test_F1": [None] * len(model_results) + [test_metrics["f1"]],
        "Test_Accuracy": [None] * len(model_results) + [test_metrics["accuracy"]],
        "Test_AUC": [None] * len(model_results) + [test_metrics["auc"]],
    }
)

print("Comparación Final de Modelos:")
print(final_comparison.round(4))

## 11. Guardar Modelo Final

In [ ]:
print("\n💾 GUARDANDO MODELO FINAL")
print("=" * 25)

# Crear directorio si no existe
os.makedirs("../models", exist_ok=True)

# Guardar modelo optimizado
model_path = f'../models/best_model_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model_tuned, model_path)
print(f"✅ Modelo guardado: {model_path}")

# Guardar métricas
metrics_path = "../models/model_metrics.json"
import json

all_metrics = {
    "best_model": best_model_name,
    "best_params": grid_search.best_params_,
    "base_models_comparison": model_results,
    "cv_results": cv_results,
    "final_test_metrics": test_metrics,
}

with open(metrics_path, "w") as f:
    # Convertir numpy types a tipos nativos de Python
    def convert_numpy(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return obj

    # Aplicar conversión recursivamente
    def clean_dict(d):
        if isinstance(d, dict):
            return {k: clean_dict(v) for k, v in d.items()}
        elif isinstance(d, list):
            return [clean_dict(item) for item in d]
        else:
            return convert_numpy(d)

    clean_metrics = clean_dict(all_metrics)
    json.dump(clean_metrics, f, indent=2)

print(f"✅ Métricas guardadas: {metrics_path}")

## 12. Insights y Conclusiones

In [ ]:
print("\n🎯 INSIGHTS Y CONCLUSIONES")
print("=" * 35)

insights = [
    f"🏆 Mejor modelo: {best_model_name} con F1={test_metrics['f1']:.4f}",
    f"📊 Accuracy final en test: {test_metrics['accuracy']:.4f} ({test_metrics['accuracy']*100:.1f}%)",
    f"🎯 AUC-ROC: {test_metrics['auc']:.4f} (excelente capacidad discriminativa)",
    f"⚖️ Precision: {test_metrics['precision']:.4f}, Recall: {test_metrics['recall']:.4f}",
]

performance_analysis = []
if test_metrics["accuracy"] > 0.80:
    performance_analysis.append("✅ Objetivo de >80% accuracy ALCANZADO")
else:
    performance_analysis.append(
        f"⚠️ Accuracy {test_metrics['accuracy']*100:.1f}% por debajo del objetivo 80%"
    )

if test_metrics["auc"] > 0.85:
    performance_analysis.append("✅ Excelente capacidad discriminativa (AUC > 0.85)")
elif test_metrics["auc"] > 0.75:
    performance_analysis.append("✅ Buena capacidad discriminativa (AUC > 0.75)")

print("RESULTADOS PRINCIPALES:")
for insight in insights:
    print(f"  {insight}")

print("\nANÁLISIS DE PERFORMANCE:")
for analysis in performance_analysis:
    print(f"  {analysis}")

# Análisis de overfitting
train_val_diff = tuned_metrics["train_accuracy"] - tuned_metrics["val_accuracy"]
if train_val_diff > 0.05:
    print(f"\n⚠️ POSIBLE OVERFITTING:")
    print(f"  Diferencia Train-Val: {train_val_diff:.4f}")
    print(f"  Train Accuracy: {tuned_metrics['train_accuracy']:.4f}")
    print(f"  Val Accuracy: {tuned_metrics['val_accuracy']:.4f}")
else:
    print(f"\n✅ BUEN BALANCE BIAS-VARIANCE:")
    print(f"  Diferencia Train-Val: {train_val_diff:.4f} (< 0.05)")

## 13. Próximos Pasos

In [ ]:
print("\n🚀 PRÓXIMOS PASOS")
print("=" * 18)

next_steps = [
    "Interpretación de Resultados (05_model_evaluation.ipynb)",
    "- Análisis detallado de casos mal clasificados",
    "- Interpretación de feature importance en contexto histórico",
    "- Validación de insights con conocimiento del dominio",
    "- Análisis de bias en predicciones por subgrupos",
    "",
    "Storytelling Final",
    "- Conexión de resultados ML con narrativa histórica",
    "- Validación de hipótesis iniciales",
    "- Insights para protocolo de emergencias modernas",
    "- Visualizaciones finales para presentación",
    "",
    "Documentación Completa",
    "- Actualizar development log con challenges de modelado",
    "- Completar reporte técnico final",
    "- Preparar presentación ejecutiva",
]

for step in next_steps:
    if step == "":
        print()
    elif step.startswith("-"):
        print(f"  {step}")
    else:
        print(f"• {step}")

print(f"\n✅ ¡Modelado completado exitosamente!")
print(
    f"🎯 Modelo final: {best_model_name} con {test_metrics['accuracy']*100:.1f}% accuracy"
)
print(f"📊 Listo para interpretación y storytelling final")